# Preprocessamento por mandato
Existiram 6 mandatos de 1998 a 2022

In [3]:
from tqdm import tqdm
import itertools
import pandas as pd
import numpy as np
import networkx as nx

In [4]:
anos = range(1998,2023)
diretorio_dados = "dados/cru"
diretorio_dados_processados = "dados/processado"
caminho_dados_autores = diretorio_dados + '/proposicoesAutores-{ano}.csv'
filtro_colunas_autores = ['idProposicao', 'idDeputadoAutor', 'nomeAutor', 'siglaPartidoAutor', 'siglaUFAutor']

In [7]:
def filtrar_e_preprocessar_df_autores(df_cru, ano):
    df_cru = df_cru[filtro_colunas_autores]
    df_cru.loc[:, 'ano'] = ano
    df_cru.loc[:, 'nomeAutor'] = df_cru.loc[:, 'nomeAutor'].str.upper()
    df_cru.loc[:, 'siglaPartidoAutor'] = df_cru.loc[:, 'siglaPartidoAutor'].str.upper()
    df_cru.loc[:, 'siglaUFAutor'] = df_cru.loc[:, 'siglaUFAutor'].str.upper()
    df_cru = df_cru[df_cru['idDeputadoAutor'].notna()]
    
    return df_cru

def remover_projetos_com_um_autor(df):
    df_aux = df.groupby('idProposicao').size()
    df = df[df['idProposicao'].isin(df_aux[df_aux > 1].index)]
    
    return df

def formar_arestas(df):
    df_index = df.set_index(['idProposicao', 'idDeputadoAutor'])

    dict_edges = {}
    for date, new_df in tqdm(df_index.groupby(level=0)):
        for tup in itertools.combinations([j for (i,j) in new_df.index], 2):
            t = tup
            #ordenar
            if(t[0] > t[1]):
                t = (t[1], t[0])
            if(t in dict_edges):
                dict_edges[t].append(date)
            else:
                dict_edges[t] = [date]

    df_edges = pd.DataFrame([[key[0], 
                          key[1], 
                          ','.join(map(str,dict_edges[key])), 
                          len(dict_edges[key])] for key in dict_edges],
                       columns=['no1','no2','projetos', 'weight'])
    
    return nx.from_pandas_edgelist(df_edges, source='no1', target='no2', edge_attr=True)

def iteravel_para_string(iteravel):
    lista = list(iteravel)
    if(len(lista) == 1):
        return str(lista[0])
    if(np.nan in lista):
        lista.remove(np.nan)
    return ','.join(map(str,lista))

def autores_atributos(df, anos):
    df = df[df['ano'].isin(anos)]
    df.set_index('idDeputadoAutor')
    
    info_autores = []
    for idAutor, new_df in tqdm(df.groupby(level=0)):
        info = []
        info.append(idAutor)
        info.append(new_df.iloc[0, 1])
        partido = set()
        uf = set()
        for index, row in new_df.iterrows():
            partido.add(row['siglaPartidoAutor'])
            uf.add(row['siglaUFAutor'])
        info.append(iteravel_para_string(partido))
        info.append(iteravel_para_string(uf))
        info_autores.append(info)
    
    df_deputados = pd.DataFrame(info_autores, columns=['id', 'nome', 'partidos', 'uf'])

    return df_deputados.set_index('id').to_dict('index')

In [8]:
grafos_array = []
for i in range(1999, 2023, 4):
    df_array_autores = []
    for ano in range(i,i+4):
        df = pd.read_csv(caminho_dados_autores.format(ano=ano), sep=';')
        df = filtrar_e_preprocessar_df_autores(df, ano)
        df = remover_projetos_com_um_autor(df)
        df_array_autores.append(df)
    
    df = pd.concat(df_array_autores)
    gf = formar_arestas(df)
    nx.set_node_attributes(gf, autores_atributos(df, range(i,i+4)))
    grafos_array.append(gf)
    nx.write_gml(gf, "{}/arestas{}-{}.gml".format(diretorio_dados_processados,i, i+4))
    

100%|█████████████████████████████████| 108713/108713 [01:11<00:00, 1519.13it/s]


In [15]:
for gf, i in zip(grafos_array, range(1999, 2023, 4)): 
    grau_medio = pd.DataFrame(gf.degree())[1].sum() / gf.number_of_nodes()
    print("--------------", "{}-{}:".format(i,i+3), "--------------")
    print("nos:", gf.number_of_nodes())
    print("arestas:", gf.number_of_edges())
    print("grau médio", grau_medio)
    print()
    

-------------- 1999-2002: --------------
nos: 421
arestas: 7868
grau médio 37.37767220902613

-------------- 2003-2006: --------------
nos: 624
arestas: 177859
grau médio 570.0608974358975

-------------- 2007-2010: --------------
nos: 612
arestas: 156108
grau médio 510.15686274509807

-------------- 2011-2014: --------------
nos: 638
arestas: 151916
grau médio 476.2257053291536

-------------- 2015-2018: --------------
nos: 626
arestas: 162512
grau médio 519.2076677316294

-------------- 2019-2022: --------------
nos: 611
arestas: 158359
grau médio 518.3600654664484

